In [5]:
import pandas as pd
import torch
from torch.utils.data import DataLoader, TensorDataset
from transformers import BertTokenizer, BertForSequenceClassification

In [6]:
print(torch.cuda.is_available())

False


In [7]:


# def load_model_checkpoint(model_path):
#     # Load the model checkpoint, specifying map_location='cpu' for CPU-only machines
#     checkpoint = torch.load(model_path, map_location='cpu')

#     # Initialize the model architecture
#     model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

#     # Load the state dictionary into the model
#     model.load_state_dict(checkpoint)

#     return model

# def test_model_and_save(model_path, test_dataset_path, output_file):
#     # Load the trained BERT model on a CPU-only machine
#     model = load_model_checkpoint(model_path)
#     model.eval()  # Set the model to evaluation mode

#     # Load the test dataset
#     df_test = pd.read_csv(test_dataset_path, sep='\t')

#     # Tokenize input texts and prepare input tensors
#     tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
#     tokenized_texts = df_test['text_en'].apply(lambda x: tokenizer.encode(x, add_special_tokens=True, truncation=True, max_length=256))
#     max_len = max(len(tokens) for tokens in tokenized_texts)
#     padded_sequences = [tokens + [0]*(max_len - len(tokens)) for tokens in tokenized_texts]
#     input_ids = torch.tensor(padded_sequences)

#     # Create DataLoader for inference
#     dataset = TensorDataset(input_ids)
#     batch_size = 16
#     test_loader = DataLoader(dataset, batch_size=batch_size)

#     predictions = []
#     with torch.no_grad():
#         for batch in test_loader:
#             batch_input_ids = batch[0]
#             outputs = model(batch_input_ids)
#             logits = outputs.logits
#             _, predicted = torch.max(logits, dim=1)
#             predictions.extend(predicted.tolist())

#     # Prepare output DataFrame with 'id' and 'label' columns
#     output_df = pd.DataFrame({'id': df_test['id'], 'label': predictions})

#     # Save output to TSV file without header
#     output_df.to_csv(output_file, sep='\t', index=False, header=False)

#     print(f"Test predictions saved to '{output_file}'")

# #Example usage:
# model_path = 'power_train.pth'  # Path to your trained model checkpoint
# test_dataset_path = 'D:\clef\ideology-power-st-testset\power\power-at-test.tsv'  # Path to your test dataset TSV file
# output_file = 'team-power-at-runname.tsv'  # Path to save the test predictions in TSV format

# test_model_and_save(model_path, test_dataset_path, output_file)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.


Test predictions saved to 'team-orientation-at-runname.tsv'


In [3]:


def load_model_checkpoint(model_path, device):
    # Load the model checkpoint, specifying map_location to load onto the specified device
    checkpoint = torch.load(model_path, map_location=device)

    # Initialize the model architecture
    model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

    # Load the state dictionary into the model
    model.load_state_dict(checkpoint)

    # Move the model to the specified device (GPU or CPU)
    model.to(device)

    return model

def test_model_and_save(model_path, test_dataset_path, output_file, device):
    # Determine the device to use (GPU or CPU)
    if torch.cuda.is_available() and device == 'cuda':
        device = torch.device('cuda')
    else:
        device = torch.device('cpu')

    # Load the trained BERT model
    model = load_model_checkpoint(model_path, device)
    model.eval()  # Set the model to evaluation mode

    # Load the test dataset
    df_test = pd.read_csv(test_dataset_path, sep='\t')

    # Tokenize input texts and prepare input tensors
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    tokenized_texts = df_test['text_en'].apply(lambda x: tokenizer.encode(x, add_special_tokens=True, truncation=True, max_length=256))
    max_len = max(len(tokens) for tokens in tokenized_texts)
    padded_sequences = [tokens + [0]*(max_len - len(tokens)) for tokens in tokenized_texts]
    input_ids = torch.tensor(padded_sequences)

    # Move input tensors to the specified device (GPU or CPU)
    input_ids = input_ids.to(device)

    # Create DataLoader for inference
    dataset = TensorDataset(input_ids)
    batch_size = 16
    test_loader = DataLoader(dataset, batch_size=batch_size)

    predictions = []
    with torch.no_grad():
        for batch in test_loader:
            batch_input_ids = batch[0].to(device)
            outputs = model(batch_input_ids)
            logits = outputs.logits
            _, predicted = torch.max(logits, dim=1)
            predictions.extend(predicted.cpu().tolist())  # Move predictions back to CPU

    # Prepare output DataFrame with 'id' and 'label' columns
    output_df = pd.DataFrame({'id': df_test['id'], 'label': predictions})

    # Save output to TSV file without header
    output_df.to_csv(output_file, sep='\t', index=False, header=False)

    print(f"Test predictions saved to '{output_file}'")

# Example usage:

model_path = 'power_train.pth'  # Path to your trained model checkpoint
test_dataset_path = 'D:\clef\ideology-power-st-testset\power\power-at-test.tsv'  # Path to your test dataset TSV file
output_file = 'team-power-at-runname.tsv'  # Path to save the test predictions in TSV format
device = 'cuda' if torch.cuda.is_available() else 'cpu'  # Use GPU if available, otherwise use CPU

# Test the trained model and save predictions to a TSV file
test_model_and_save(model_path, test_dataset_path, output_file, device)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


RuntimeError: Attempting to deserialize object on a CUDA device but torch.cuda.is_available() is False. If you are running on a CPU-only machine, please use torch.load with map_location=torch.device('cpu') to map your storages to the CPU.